In [12]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip

--2023-05-30 06:32:10--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.15MB/s    in 2m 42s  

2023-05-30 06:34:53 (5.09 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [13]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import pickle
import nltk

from nltk.corpus import stopwords
stopwords = stopwords.words('english')

import spacy
nlp = spacy.load("en_core_web_sm")

import gensim
import re
import pickle

from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# !pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.7 MB/s eta 0:00:00


In [3]:
df  = pd.read_csv('/content/training_set_rel3.tsv', sep='\t', encoding = "ISO-8859-1")
# df  = pd.read_csv('/content/Cleaned_Data.csv')


In [4]:
df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['essay'], df['domain1_score'], test_size=0.2, random_state=42)


In [7]:
X_train.shape

(10380,)

In [8]:
# Preprocess the text data using GloVe embeddings
embedding_dim = 100
vocab_size = 10000

In [14]:
glove_file = '/content/glove.6B.100d.txt'
embedding_index = {}
with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

In [16]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

In [17]:
# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


In [18]:
# Pad sequences to the same length
max_length = max([len(seq) for seq in X_train_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length)

In [19]:
len(tokenizer.word_index)

36434

In [20]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [21]:
# Create the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(GRU(100, return_sequences=True))
model.add(GRU(64))
model.add(Dense(1))

In [22]:
import tensorflow as tf

def r2(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return (1 - SS_res / (SS_tot + tf.keras.backend.epsilon()))

In [23]:
model.compile(loss='mean_squared_error', optimizer=RMSprop(lr=0.0006), metrics=['mse', r2])


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [24]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [25]:
history = model.fit(X_train_padded, y_train, batch_size=128, epochs=100, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/100
65/65 [==============================] - 16s 117ms/step - loss: 76.1091 - mse: 76.1091 - r2: 0.0584 - val_loss: 58.2214 - val_mse: 58.2214 - val_r2: 0.2835
Epoch 2/100
65/65 [==============================] - 6s 90ms/step - loss: 50.2767 - mse: 50.2767 - r2: 0.3795 - val_loss: 41.1221 - val_mse: 41.1221 - val_r2: 0.5006
Epoch 3/100
65/65 [==============================] - 6s 91ms/step - loss: 40.1575 - mse: 40.1575 - r2: 0.5096 - val_loss: 33.4255 - val_mse: 33.4255 - val_r2: 0.5968
Epoch 4/100
65/65 [==============================] - 6s 90ms/step - loss: 32.3292 - mse: 32.3292 - r2: 0.5993 - val_loss: 27.3697 - val_mse: 27.3697 - val_r2: 0.6740
Epoch 5/100
65/65 [==============================] - 6s 92ms/step - loss: 26.1036 - mse: 26.1036 - r2: 0.6845 - val_loss: 23.7423 - val_mse: 23.7423 - val_r2: 0.7143
Epoch 6/100
65/65 [==============================] - 6s 91ms/step - loss: 21.4680 - mse: 21.4680 - r2: 0.7351 - val_loss: 18.9447 - val_mse: 18.9447 - val_r2: 0.7722
Ep

In [26]:
# Evaluate the model
y_pred = model.predict(X_test_padded)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared:", r2_score(y_test, y_pred))

82/82 [==============================] - 3s 25ms/step
Mean Squared Error: 3.208514533305638
R-squared: 0.9587901128119531
